In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Nov 25 03:39:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
%cd /content/drive/MyDrive/Colab_Projects/21_F_자연어처리_이병준
!ls

/content/drive/MyDrive/Colab_Projects/21_F_자연어처리_이병준
 Customized_Model_Dong.ipynb   Original_Implementation.ipynb
'Datasets_&_Weight'	       Original_Model_Res.ipynb


In [4]:
from fastai import *
from fastai.text import *
from nltk.tokenize import word_tokenize
from fastai.callbacks import *
import torch.utils.data as data_utils

!pip install import_ipynb

import import_ipynb
from Original_Model_Res import *

  Using cached import_ipynb-0.1.3-py3-none-any.whl
importing Jupyter notebook from Original_Model_Res.ipynb


In [5]:
defaults.device = torch.device('cuda')

In [6]:
PADDING = 64  # 64 for Headlines, 128 for Reddit Main/Pol
DATASET_PATH = './Datasets_&_Weight/' #Dataset folder path
DATASET = 'Headlines.csv'  #Dataset file
COL = 'headline'   #headline for Headlines, comment for Reddit Main/Pol

# Word embedding weights
WEIGHTS = './Datasets_&_Weight/Weights_glove_headlines.pkl'
EMBED_DIM = 50 # 50 for GloVe, 300 for FastText
STATIC = False #Static or non-static embeddings

In [7]:
def pad_to(x:Collection[str], pad_til = PADDING) -> Collection[str]:
    res = []
    count = 0
    for t in x:
        res.append(t)
        count += 1
    while count < pad_til:
        res.append(PAD)
        count +=1
    return res

In [8]:
tokenizer = Tokenizer(SpacyTokenizer, 'en', 
                      pre_rules=[fix_html, replace_rep, replace_wrep, spec_add_spaces, rm_useless_spaces], 
                      post_rules=[replace_all_caps, deal_caps, pad_to])

processor = [TokenizeProcessor(tokenizer=tokenizer), NumericalizeProcessor()]

In [9]:
data = (TextList.from_csv(DATASET_PATH, DATASET, cols=COL, processor=processor))

In [10]:
data = data.split_from_df(col='valid').label_from_df(cols=0).databunch()

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [11]:
weights_matrix = pickle.load(open(WEIGHTS, 'rb'))

In [12]:
net = ResNet(weights_matrix, embed_dim = EMBED_DIM, static = STATIC)

In [13]:
net.to('cuda')

ResNet(
  (embedding): Embedding(9121, 50, padding_idx=1)
  (init_conv): Conv1d(1, 16, kernel_size=(3, 50), stride=(1,), padding=(1, 0), bias=False)
  (features): Sequential(
    (init_conv): Conv1d(1, 16, kernel_size=(3, 50), stride=(1,), padding=(1, 0), bias=False)
    (init_norm): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (init_relu): LeakyReLU(negative_slope=0.01, inplace=True)
    (resblock1): Sequential(
      (0): SequentialEx(
        (layers): ModuleList(
          (0): Sequential(
            (0): Conv1d(16, 16, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
            (1): LeakyReLU(negative_slope=0.015, inplace=True)
            (2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (3): SelfAttention(
              (query): Conv1d(16, 2, kernel_size=(1,), stride=(1,), bias=False)
              (key): Conv1d(16, 2, kernel_size=(1,), stride=(1,), bias=False)
              (value): 

In [14]:
learn = Learner(data=data, model=net, wd=0.1, loss_func=CrossEntropyFlat(),  metrics=[accuracy, FBeta(average='macro',beta=1)])

In [15]:
learn.fit_one_cycle(20, 1e-03, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,f_beta,time
0,2.704025,0.745317,0.441782,0.317445,01:07
1,1.446471,0.697722,0.463684,0.412198,01:07
2,0.859853,0.693867,0.486896,0.477105,01:07
3,0.732008,0.705447,0.438974,0.305061,01:07
4,0.696503,0.702652,0.438974,0.305061,01:06
5,0.671876,0.740269,0.438974,0.305061,01:06
6,0.595625,1.117074,0.438974,0.305061,01:06
7,0.485099,0.822885,0.663422,0.649896,01:07
8,0.396136,0.740572,0.802321,0.802265,01:07
9,0.309840,0.571817,0.818982,0.818979,01:07


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
